In [ ]:
import numpy as np
from iminuit import Minuit
from iminuit.cost import BinnedNLL, UnbinnedNLL
from scipy.stats import norm
from jacobi import jacobi
from math import erf

In [ ]:
# generate some data
rng = np.random.default_rng(1)
t_mu, t_sigma = 1., 0.1
x = rng.normal(t_mu, t_sigma, 2000)
nx, xe = np.histogram(x, bins=50, range=(.5, 1.5))

In [ ]:
# define the model and cost function
#def cdf(x, mu, sigma):
#    z = (x - mu) / (np.sqrt(2) * sigma)
#    return (1 + np.vectorize(erf)(z)) * 0.5
def cdf(x, mu, sigma):
    return norm.cdf(x, loc=mu, scale=sigma)

def cdf_grad(x, *args):
    return jacobi(lambda p: cdf(x, *p), args)[0].T

def pdf(x, mu, sigma):
    return norm.pdf(x, loc=mu, scale=sigma)

def pdf_grad(x, *args):
    return jacobi(lambda p: pdf(x, *p), args)[1].T

## Binned

In [ ]:
cost = BinnedNLL(nx, xe, cdf, grad=cdf_grad)
m = Minuit(cost, mu=0, sigma=1)
m.migrad()

In [ ]:
%%timeit
m.migrad()

In [ ]:
cost = BinnedNLL(nx, xe, cdf)
m = Minuit(cost, mu=0, sigma=1)
m.migrad()

In [ ]:
%%timeit
m.migrad()

## Unbinned

In [ ]:
cost = UnbinnedNLL(x, pdf, grad=pdf_grad)
m = Minuit(cost, mu=0, sigma=1)
m.migrad()

In [ ]:
%%timeit
m.migrad()

In [ ]:
cost = UnbinnedNLL(x, pdf)
m = Minuit(cost, mu=0, sigma=1)
m.migrad()

In [ ]:
%%timeit
m.migrad()

In [ ]:
%%timeit
cost._value((1, 0.1))